# Quick Start (oda api v1.1.26)

In [ ]:
from oda_api.api import DispatcherAPI
from oda_api.plot_tools import OdaImage, OdaLightCurve
from oda_api.data_products import BinaryData
import os

from importlib import reload

from oda_api.token import discover_token

import logging
logging.getLogger('oda_api').addHandler(logging.StreamHandler())
logging.getLogger().setLevel(logging.DEBUG)

logging.getLogger().setLevel(logging.DEBUG)
logging.getLogger('oda_api').addHandler(logging.StreamHandler())

## Connection to the dispatcher 

In [ ]:
#TODO: change to prod
disp=DispatcherAPI(url='https://frontend-staging.obsuks1.unige.ch/mmoda/dispatch-data',instrument='mock')
# disp=DispatcherAPI(url='http://0.0.0.0:8001',instrument='mock')

## We upload the product to the gallery, configuring the parameters we wish to show in the gallery

### For the data product we wish to upload the following parameters can be set:

* **product_title**: title to assign to the product, in case this is not provided one is automatically built using the name of the source and the type of product
* **observation**: this can be indicated in two different ways
 * by specifying the id of an already present observation (eg 'test observation')
 * by specifying the time range, in particular the value of `T1` and `T2` in the following format `'2003-03-15T23:27:40.0'`
* **produced_by**: direct link to the notebook used to generated the product
* **gallery_image_path**: path of the generated image and to be uploaded over the gallery
* **fits_file_path**: a list of fits file links used for the generation of the product to upload over the gallery
* **token**: user token
* **insert_new_source**: a boolean value to specify if, in case a source currently not available on the product gallery is passed within the parameters, this will be created and then used for the newly created data product
* **validate_source**: a boolean value to specify if, in case a source is passed within the parameters, this will be validated against an online service
* **force_insert_not_valid_new_source**: a boolean value to specify if, in case a source is passed within the parameters and its validation fails, this should be provided as a parameter for the data product
* **apply_fields_source_resolution**: a boolean value to specify if, in case a source is passed within the parameters and then successfully validated, to apply the parameters values returned from the validation (an example of these parameters are RA and DEC)
* **kwargs**: keyword arguments representing the parameters values used to generate the product. Amongst them it is important to mention the following
 * **instrument**: name of the instrument used for the generated product (e.g. isgri, jemx1)
 * **product_type**: type of product generated (e.g. isgri_lc, jemx_image)
 * **src_name**: name of a known source (eg Crab, Cyg X-1)
 * **others**: other parameters used for the product (e.g. RA=25). Not all the parameters are currently supported, but the list of the supported ones will be extended

#### Let's get the token

In [ ]:
token = discover_token()

### Example of an image product

In [ ]:
tstart='2003-03-15T23:27:40.0'
tstop='2023-03-16T00:03:12.0'
source_name='OAO 1657-415'
E1_keV=28.
E2_keV=50.
RA=50
DEC=45
detection_threshold=5.0

instrument='isgri'
product_type='isgri_image'

data_collection=disp.get_product(instrument=instrument,
                                 product=product_type,
                                 T1='2003-03-15T23:27:40.0',
                                 T2='2023-03-16T00:03:12.0',
                                 integral_data_rights="public",
                                 osa_version='OSA10.2',
                                 RA=50,
                                 DEC=45,
                                 max_pointings=1,
                                 detection_threshold=detection_threshold,
                                 radius=15.,
                                 product_type='Dummy',
                                 token=token
                                )

In [ ]:
image_product = OdaImage(data_collection)
img_fn = image_product.get_image_for_gallery()

# generate two dummy fits files
fits_file_fn_1 = image_product.write_fits(file_prefix='first')
fits_file_fn_2 = image_product.write_fits(file_prefix='second')
 
# token = discover_token()

insert_new_source=False
force_insert_not_valid_new_source=False
validate_source=True
apply_fields_source_resolution=True

d = disp.post_data_product_to_gallery(token=token,
                                      RA=55, DEC=46, e1_kev=20, e2_kev=40,
                                      instrument=instrument, product_type=product_type,
                                      product_title="very nice title",
                                      src_name=source_name,
                                      validate_source=validate_source,
                                      insert_new_source=insert_new_source,
                                      apply_fields_source_resolution=apply_fields_source_resolution,
                                      force_insert_not_valid_new_source=force_insert_not_valid_new_source,
                                      gallery_image_path=img_fn,
                                      fits_file_path=[fits_file_fn_1, fits_file_fn_2]
                                     )

### Example of a light curve product

In [ ]:
tstart='2021-02-01T00:00:00'
tstop='2021-02-27T23:59:59'
source_name='OAO 1657-415'
E1_keV=28.
E2_keV=50.
RA=263
DEC=-24.7456
detection_threshold=7.0

product_type='isgri_lc'

data_collection=disp.get_product(instrument=instrument,
                                 product=product_type,
                                 T1=tstart,
                                 T2=tstop,
                                 osa_version='OSA11.1',
                                 src_name=source_name,
                                 RA=RA,
                                 DEC=DEC,
                                 E1_keV=E1_keV,
                                 E2_keV=E2_keV,
                                 integral_data_rights='public',
                                 detection_threshold=detection_threshold,
                                 off_line='False',
                                 radius=8.,
                                 product_type='Dummy'
                                )

In [ ]:
image_product = OdaLightCurve(data_collection)
img_fn = image_product.get_image_for_gallery( in_source_name=source_name)

fits_file_fn = image_product.write_fits(source_name=source_name)[0]

notebook_link='https://github.com/oda-hub/oda_api/blob/master/doc/source/user_guide/UploadToGallery.ipynb'

insert_new_source=False
force_insert_not_valid_new_source=False
validate_source=True
apply_fields_source_resolution=True
                                             
d = disp.post_data_product_to_gallery(token=token,
                                      RA=RA, DEC=DEC, e1_kev=E1_keV, e2_kev=E2_keV,
                                      product_title=source_name,
                                      gallery_image_path=img_fn,
                                      fits_file_path=[fits_file_fn],
                                      src_name=source_name,
                                      validate_source=validate_source,
                                      insert_new_source=insert_new_source,
                                      apply_fields_source_resolution=apply_fields_source_resolution,
                                      force_insert_not_valid_new_source=force_insert_not_valid_new_source,
                                      instrument=instrument,
                                      product_type=product_type,
                                      produced_by=notebook_link,
                                      observation_id=source_name + '_' + tstart + '_' + tstop
                                     )